In [1]:
# imports
import sys

sys.path.append("..")
from src.utils import *
import polars as pl
from datetime import datetime as dt
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind, permutation_test

In [2]:
# polars config
pl.Config.load_from_file("../polars_cfg.json")

In [3]:
# data file paths
clean_summary_path = "../data/clean_summary_data_merged.csv"
clean_incomplete_path = "../data/clean_incomplete_data_merged.csv"
clean_all_path = "../data/clean_all_data_merged.csv"

In [4]:
df_summary = pl.read_csv(clean_summary_path)
df_incomplete = pl.read_csv(clean_incomplete_path)
df_incomplete_summary = get_summary(df_incomplete).select(
    df_summary.drop("condition").columns
)
df_summary_all, _ = include_incomplete(df_summary, df_incomplete)

df_all = pl.read_csv(clean_all_path)

# parse time
df_all = df_all.with_columns(
    pl.col("time").map_elements(parse_mixed_datetime, return_dtype=pl.Datetime)
)
df_incomplete = df_incomplete.with_columns(
    pl.col("time").map_elements(parse_mixed_datetime, return_dtype=pl.Datetime)
)

Filtered out 59/783 incomplete participant task attempts
Filtered out 59/783 incomplete participant task attempts


In [5]:
df_by_participant = df_summary_all.group_by("hashed_id").agg(
    pl.max("task_number").alias("tasks_completed"),
    pl.min("task_number").alias("first_task"),
    pl.first("task_type"),
    pl.first("exp_name"),
)

# number of participants given 5 tasks on training set
five = df_by_participant.filter(
    (pl.col("task_type") == "training")
    & ~(pl.col("exp_name").is_in(["expv0", "expv1"]))
)
ten = df_by_participant.filter(
    (pl.col("task_type") == "training") & (pl.col("exp_name").is_in(["expv0", "expv1"]))
)
print("Number of participants given 5 tasks on training set: ", len(five))
print("Number of participants given 10 tasks on training set: ", len(ten))

Number of participants given 5 tasks on training set:  542
Number of participants given 10 tasks on training set:  241


In [6]:
# get number of participants who were prevented from submitting the same task
num_copy_allowed = len(
    df_all.group_by("hashed_id")
    .agg(pl.max("time").alias("last_time"), pl.first("task_type"))
    .filter((pl.col("last_time") < dt(2023, 12, 1, 15, 43, 10)))
)
total = len(df_summary["hashed_id"].unique())
print(
    f"Number of participants not prevented from submitting the same grid after an incorrect attempt: {num_copy_allowed}/{total}\n"
)

Number of participants not prevented from submitting the same grid after an incorrect attempt: 340/1425



In [18]:
# rate of copied outputs
test_outputs = (
    df_summary.filter(~pl.col("solved"))
    .group_by("joint_id_task")
    .agg(pl.col("test_output_grid"), pl.len())
)
total = test_outputs.select(pl.sum("len")).item()
# apply set to remove duplicates
test_outputs = test_outputs.with_columns(
    pl.col("test_output_grid").list.n_unique().alias("unique_count")
)
num_copied = (
    test_outputs.select((pl.col("len") - pl.col("unique_count")).alias("num_copies"))
    .select(pl.sum("num_copies"))
    .item()
)
print(
    f"Proportion of incorrect submissions that were copies of previous submissions outputs: {num_copied/total:.2f}\n"
)

Proportion of incorrect submissions that were copies of previous submissions outputs: 0.07



In [19]:
# top ten hardest tasks ordered by number of participants
df_summary.filter(pl.col("task_type") == "evaluation").select(
    pl.col(["joint_id_task", "task_name", "attempt_number", "solved"])
    .top_k_by("attempt_number", k=1)
    .over("joint_id_task", mapping_strategy="explode")
).group_by("task_name").agg(pl.sum("solved") / pl.len(), pl.len().alias("count")).sort(
    ["solved", "count"], descending=[False, True]
).head(
    10
)

task_name,solved,count
str,f64,u32
"""31d5ba1a.json""",0.0,9
"""b4a43f3b.json""",0.0,9
"""a8610ef7.json""",0.0,6
"""79fb03f4.json""",0.0,6
"""4ff4c9da.json""",0.0,4
"""40f6cd08.json""",0.0,2
"""8719f442.json""",0.0,1
"""fea12743.json""",0.090909,11
"""e6de6e8f.json""",0.111111,9
